In [ ]:
def Produce_Storage_NetLoads(PV_a,Annual_Demand_end,capacity_a,H2,NH3):

    eff_h2 = H2.eff_h2
    p_cap_h2 = H2.p_cap_h2
    e_cap_h2 = H2.e_cap_h2
    h2_soc_start = H2.h2_soc_start

    eff_nh3 = NH3.eff_nh3
    p_cap_nh3 = NH3.p_cap_nh3
    e_cap_nh3 = NH3.e_cap_nh3
    nh3_soc_start = NH3.nh3_soc_start

    #daily demand in Wh; this is the electrical energy needed by the trains every day
    daily_demand_end = Annual_Demand_end/365.25

    #print(Annual_Demand_end)
    #print(daily_demand_end)

    #turn into a vector of the same length as the daily df
    NDays = len(PV_Epr1MW.df_day.index)
    Daily_Demand_End_Vector = np.zeros(NDays)

    for i in range(0,NDays):
        Daily_Demand_End_Vector[i] = daily_demand_end
        i+=1

    Daily_Demand_End_List = list(Daily_Demand_End_Vector)

    #lump into one dataframe
    #PV_Epr1MW_supp_demand = PV_Epr1MW.df_day
    #PV_Epr1MW_supp_demand['Daily Demand (kWh)']=Daily_Demand_End_List
    #PV_Epr1MW_supp_demand.plot()

    #We are now going to model storage of Hydrogen and Ammonia separately, using an initial power capacity of 23MW for our solar farm. The amount of solar electricity going to hydrogen storage will at most be enough to supply the following day's daily demand (assumed constant each day). If there is any surplus, it will go to ammonia storage.

    #turn farm into a class
    PV_a_supp_demand = PV_a.df_day
    PV_a_supp_demand['Daily Demand (kWh)']=Daily_Demand_End_List
    PV_a_supp_demand['Nondispatchable Net Load (kWh)']=PV_a_supp_demand['Daily Demand (kWh)'] - PV_a_supp_demand['electricity (kWh)']
    #PV_a_supp_demand.plot()

    #shorthand for the nondispatchable net load, put in list for ease
    nd_nl = PV_a_supp_demand['Nondispatchable Net Load (kWh)'].values.tolist()

    outputi_h2 = np.zeros((NDays,1)) # storage energy flow; assume all is in ammonia for the time being
    soc_h2 = np.zeros((NDays,1))
    soc_h2[0] = h2_soc_start

    #use battery model for hydrogen
    for j in range(NDays):

        #charge in battery on 1st jan
        if j==0:
            socval = soc_h2[0]
        else:
            socval = soc_h2[j-1]

        #determine battery operation depending on nondispatchable net load
        if nd_nl[j] > 0:
            #power flow into battery is either the capacity, nondispatchable net load, or existing state of charge (if it is fully charged)
            outputi_h2[j]=min((p_cap_h2),nd_nl[j],eff_h2*socval) #amount of power flowing out of storage; * by eff due to losses
            soc_h2[j] = socval - (1/eff_h2)*outputi_h2[j]

        elif nd_nl[j] < 0:
            outputi_h2[j]=max(-p_cap_h2,nd_nl[j],-(1/eff_h2)*(e_cap_h2-socval)) #amount of power flowing into storage
            soc_h2[j] = socval - eff_h2*outputi_h2[j]

        elif nd_nl[j] == 0:
            soc_h2[j] = socval

    #calculate the net load of the final system
    net_load_h2 = nd_nl - outputi_h2
    net_load_h2 = net_load_h2[0]
    net_load_h2 = net_load_h2.tolist()

    #append net load with hydrogen storage to dataframe and plot...
    PV_a_supp_demand['Net Load with H2 only (kWh)'] = net_load_h2
    PV_a_supp_demand['H2 State of Charge (kWh)'] = soc_h2
    PV_a_supp_demand['H2 Energy Output (kWh)'] = outputi_h2
    PV_a_supp_demand['Demand met by H2 (kWh)'] = PV_a_supp_demand['Nondispatchable Net Load (kWh)'] - PV_a_supp_demand['Net Load with H2 only (kWh)']
    #PV_a_supp_demand.plot()

    #We can now put in Ammonia storage; considering that the net load with Hydrogen is now the 'nondispatchable net load' when considering ammonia storage.

    outputi_nh3 = np.zeros((NDays,1)) # storage energy flow; assume all is in ammonia for the time being
    soc_nh3 = np.zeros((NDays,1)) #storage 'state of charge' initialisation
    soc_nh3[0] = nh3_soc_start
    #use battery model for hydrogen
    for j in range(NDays):

        #charge in battery on 1st jan
        if j==0:
            socval2 = soc_nh3[0]
        else:
            socval2 = soc_nh3[j-1]

        #determine battery operation depending on nondispatchable net load
        if net_load_h2[j] > 0:
            #power flow into battery is either the capacity, nondispatchable net load, or existing state of charge (if it is fully charged)
            outputi_nh3[j]=min((p_cap_nh3),net_load_h2[j],eff_nh3*socval2) #amount of power flowing out of storage; * by eff due to losses
            soc_nh3[j] = socval2 - (1/eff_nh3)*outputi_nh3[j]

        elif net_load_h2[j] < 0:
            outputi_nh3[j]=max(-p_cap_nh3,net_load_h2[j],-(1/eff_nh3)*(e_cap_nh3-socval2)) #amount of power flowing into storage
            soc_nh3[j] = socval2 - eff_nh3*outputi_nh3[j]

        elif net_load_h2[j] == 0:
            soc_nh3[j] = socval2

    #calculate the net load of the final system
    net_load_nh3 = net_load_h2 - outputi_nh3
    net_load_nh3 = net_load_nh3[0]
    net_load_nh3 = net_load_nh3.tolist()

    #append net load with hydrogen storage to dataframe and plot...
    PV_a_supp_demand['Net Load with NH3 (kWh)'] = net_load_nh3
    PV_a_supp_demand['NH3 State of Charge (kWh)'] = soc_nh3
    PV_a_supp_demand['NH3 Energy Output (kWh)'] = outputi_nh3
    PV_a_supp_demand['Demand met by NH3 (kWh)'] = PV_a_supp_demand['Nondispatchable Net Load (kWh)'] - PV_a_supp_demand['Net Load with NH3 (kWh)']
    #PV_a_supp_demand.plot()

    #PV_a_supp_demand['electricity (kWh)'].plot(label = 'Solar Electricity (kWh)')
    #PV_a_supp_demand['Net Load with NH3 (kWh)'].plot()
    #PV_a_supp_demand['NH3 Energy Output (kWh)'].plot()
    #PV_a_supp_demand['H2 State of Charge (kWh)'].plot()
    #plt.legend(['Solar Electricity (kWh)','Net Load with NH3 (kWh)','H2 State of Charge (kWh)'])

    #PV_a_supp_demand['NH3 State of Charge (kWh)'].plot()
    #plt.legend(['NH3 State of Charge (kWh)'])

    return PV_a_supp_demand


